## Approach

Our algorithm will proceed in the following steps:


1.	[Divide cities/grids to grids/sub-grids](#1) 
2.	[Assign orders to grids](#2) 
3.	[Assign markets to grids](#3) 
4.	[Calculate the number of motors to be assigned to the grids](#4) 
5.	[Assign motors to the markets within grids](#5) 

Dividing cities to grids is being done recursively to minimize computation power.

In [1]:
import pandas as pd 
import numpy as np
from geopy import distance
from tqdm import tqdm
import math
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
city_markets = pd.read_excel('city_markets.xlsx')
city_orders = pd.read_excel('city_orders.xlsx')


In [3]:
city_markets.head()

,market_coor_x,market_coor_y,market_ID,district
0,39.115752,27.176445,1057,10004
1,39.112795,27.187978,1071,10004
2,39.111586,27.172633,1069,10004
3,39.121336,27.179932,1061,10004
4,39.118501,27.179032,1077,10004


In [4]:
city_orders.head()

,order_ID,customer_coor_x,customer_coor_y,district
0,6604496,38.325674,27.131048,10009
1,6604641,38.392046,27.083890,10016
2,6604764,38.398984,27.164191,10006
3,6604795,38.316243,27.130417,10009
4,6604891,38.499003,27.060534,10027


<a id="1"> </a>
### Divide cities/grids to grids/sub-grids 


In [5]:
latitude_start = 37.632
longitude_start = 25.956
latitude_end = 39.641
longitude_end = 28.796
motor_num = 106

In [6]:
def calculate_grid(lat_start, lat_end, lon_start,lon_end,num_vehicle):
    lat_diff = lat_end - lat_start
    lon_diff = lon_end - lon_start
    grid_area = (lat_diff*lon_diff)/num_vehicle
    grid_edge = np.sqrt(grid_area)
    lat_slide = np.round(lat_diff/grid_edge)
    lon_slide = np.round(lon_diff/grid_edge)
    lat_step_size = lat_diff/lat_slide
    lon_step_size = lon_diff/lon_slide
    return lat_slide, lon_slide,lat_step_size, lon_step_size

In [7]:
lat_slide, lon_slide, lat_step_size, lon_step_size = calculate_grid(latitude_start, latitude_end, longitude_start,longitude_end, motor_num)


In [8]:
def grid_structure(lat_start, lon_start,lat_slide, lon_slide, lat_step_size, lon_step_size):
    df_grid = pd.DataFrame()
    df_grid['grid_id'] = np.arange(lat_slide*lon_slide)
    df_grid['lat_start'] = 0
    df_grid['lat_end'] = 0
    df_grid['lon_start'] = 0
    df_grid['lon_end'] = 0
    grid_id = 0
    for i in range(lat_slide.astype(int)):
        lat_start_ = lat_start + lat_step_size*i
        lat_end_ = lat_start_ + lat_step_size
        for j in range(lon_slide.astype(int)):
            lon_start_ = lon_start + lon_step_size * j
            lon_end_ = lon_start_ + lon_step_size   
            df_grid.loc[grid_id,'lat_start'] = lat_start_
            df_grid.loc[grid_id,'lat_end'] = lat_end_
            df_grid.loc[grid_id,'lon_start'] = lon_start_
            df_grid.loc[grid_id,'lon_end'] = lon_end_
            grid_id+=1
    return df_grid       

In [9]:
main_grid_df = grid_structure(latitude_start, longitude_start, lat_slide, lon_slide, lat_step_size, lon_step_size)


In [10]:
main_grid_df.head()

,grid_id,lat_start,lat_end,lon_start,lon_end
0,0.0,37.632,37.855222,25.956000,26.192667
1,1.0,37.632,37.855222,26.192667,26.429333
2,2.0,37.632,37.855222,26.429333,26.666000
3,3.0,37.632,37.855222,26.666000,26.902667
4,4.0,37.632,37.855222,26.902667,27.139333


<a id="2"> </a>
### Assign orders to grids


In [11]:
def order_to_grid(df_order, df_grid):
    for idx in df_order.index:
        tempID = df_grid[(df_order.loc[idx,'customer_coor_x']>=df_grid.lat_start) & (df_order.loc[idx,'customer_coor_x']<df_grid.lat_end) & (df_order.loc[idx,'customer_coor_y']>=df_grid.lon_start) & (df_order.loc[idx,'customer_coor_y']<df_grid.lon_end)].grid_id
        if len(tempID) != 0:
            df_order.loc[idx,'grid_id'] = tempID.values[0]
        else: 
            df_order.loc[idx,'grid_id'] = 'out_of_scope'
        
    return df_order       

In [12]:
city_orders = order_to_grid(city_orders, main_grid_df) 

In [13]:
#Eliminate out of scope orders
city_orders.drop(axis=0, index=city_orders[city_orders.grid_id == 'out_of_scope'].index, inplace=True)


<a id="3"> </a>
### Assign markets to grids 


In [14]:
def market_to_grid(df_market, df_grid):
    for idx in df_market.index:
        tempID = df_grid[(df_market.loc[idx,'market_coor_x']>df_grid.lat_start) & (df_market.loc[idx,'market_coor_x']<df_grid.lat_end) & (df_market.loc[idx,'market_coor_y']>df_grid.lon_start) & (df_market.loc[idx,'market_coor_y']<df_grid.lon_end)].grid_id
        if len(tempID) != 0:
            df_market.loc[idx,'grid_id'] = tempID.values[0] 
        else: 
            df_market.loc[idx,'grid_id'] = 100
    return df_market        

In [15]:
city_markets = market_to_grid(city_markets,main_grid_df)

<a id="4"> </a>
### Calculate the number of motors to be assigned to the grids 


In [16]:
# Number of orders per grid
grid_order = city_orders.groupby(['grid_id']).agg({'order_ID':'count'}).reset_index()
grid_order.rename(columns={'order_ID':'order_count'},inplace=True)
main_grid_df_ = main_grid_df.merge(grid_order, on='grid_id' , how='left')
main_grid_df_['order_count'].fillna(0,inplace=True)


In [17]:
# Number of markets per grid
grid_market_count = city_markets.groupby(['grid_id']).agg({'market_ID':'count'}).reset_index()
grid_market_count.rename(columns={'market_ID':'market_count'},inplace=True)
main_grid_df_ = main_grid_df_.merge(grid_market_count,on='grid_id' , how='left')
main_grid_df_['market_count'].fillna(0,inplace=True)

In [18]:
main_grid_df_ = main_grid_df_[(main_grid_df_.order_count != 0)]
main_grid_df_.reset_index(inplace=True,drop=True)
main_grid_df_.head(10)

,grid_id,lat_start,lat_end,lon_start,lon_end,order_count,market_count
0,7.0,37.632000,37.855222,27.612667,27.849333,1.0,0.0
1,16.0,37.855222,38.078444,26.902667,27.139333,75.0,10.0
2,17.0,37.855222,38.078444,27.139333,27.376000,106.0,7.0
3,18.0,37.855222,38.078444,27.376000,27.612667,35.0,0.0
4,25.0,38.078444,38.301667,26.192667,26.429333,138.0,5.0
5,27.0,38.078444,38.301667,26.666000,26.902667,275.0,14.0
6,28.0,38.078444,38.301667,26.902667,27.139333,524.0,12.0
7,29.0,38.078444,38.301667,27.139333,27.376000,815.0,30.0
8,31.0,38.078444,38.301667,27.612667,27.849333,276.0,16.0
9,32.0,38.078444,38.301667,27.849333,28.086000,202.0,11.0


In [19]:
def assign_motor(df_grid, motor_count):
    order_mean = df_grid.order_count.sum()/len(df_grid)
    df = df_grid.copy()
    df_assigned = []
    df['motor_count'] = 0
    while motor_count>0:
        df.sort_values(by='order_count',inplace=True, ascending=False)
        df.reset_index(inplace=True, drop=True)
        if df.loc[0,'motor_count'] < df.loc[0,'market_count']:
            df.loc[0,'motor_count'] += 1
            df.loc[0,'order_count'] -= order_mean
            motor_count-=1
        else: #motor count is equal market count but order count is still high
            df_assigned.append(df.iloc[0])
            df = df.iloc[1:]
    df_assigned_df = pd.DataFrame(df_assigned) 
    df = pd.concat([df_assigned_df,df],ignore_index=True)
    return df

In [20]:
main_grid_motor_assign = assign_motor(main_grid_df_, motor_num)
main_grid_motor_assign.head()

,grid_id,lat_start,lat_end,lon_start,lon_end,order_count,market_count,motor_count
0,18.0,37.855222,38.078444,27.376000,27.612667,35.00,0.0,0.0
1,7.0,37.632000,37.855222,27.612667,27.849333,1.00,0.0,0.0
2,50.0,38.524889,38.748111,26.429333,26.666000,-1022.36,1.0,1.0
3,43.0,38.301667,38.524889,27.612667,27.849333,-1047.36,1.0,1.0
4,33.0,38.078444,38.301667,28.086000,28.322667,-2075.72,2.0,2.0


In [21]:
main_grid_motor_assign.grid_id = main_grid_motor_assign.grid_id.astype(str)
city_markets.grid_id = city_markets.grid_id.astype(str) 
city_orders.grid_id = city_orders.grid_id.astype(str)  
main_grid_df_.grid_id = main_grid_df_.grid_id.astype(str)

In [22]:
city_market_grid = city_markets.merge(main_grid_motor_assign[['grid_id','motor_count','market_count']],on='grid_id',how='left')


In [23]:
city_market_grid.drop_duplicates(subset='grid_id',keep='first')['motor_count'].sum()

106.0

### Divide grids to subgrids - recursive 

In [24]:
## If a grid combination is higher than 500 
def grid_division(main_grid_motor_assign, market_df, order_df, idx_to_drop):
    main_grid_motor_assign['upper_id'] = 0
    for idx in main_grid_motor_assign.index:
        combination_count = math.comb(int(main_grid_motor_assign.loc[idx,'market_count']),int(main_grid_motor_assign.loc[idx,'motor_count']))
        sub_grid_split = []
        sub_grid_market = []
        if combination_count > 500: 
            sub_grid_split.append(main_grid_motor_assign.loc[idx,:])
            new_df = pd.DataFrame(sub_grid_split)
            idx_to_drop.append(new_df.loc[idx,'grid_id'])
            reassigned_ID=[]
            for idx_ in new_df.index:
                new_df.loc[idx_,['lat_slide','lon_slide','lat_step_size','lon_step_size']] = calculate_grid(new_df.loc[idx_,'lat_start'], new_df.loc[idx_,'lat_end'], new_df.loc[idx_,'lon_start'],new_df.loc[idx_,'lon_end'], new_df.loc[idx_,'motor_count'])
                temp_df = grid_structure(new_df.loc[idx_,'lat_start'],new_df.loc[idx_,'lon_start'],new_df.loc[idx_,'lat_slide'], new_df.loc[idx_,'lon_slide'],new_df.loc[idx_,'lat_step_size'],new_df.loc[idx_,'lon_step_size'])
                temp_df['upper_id'] = new_df.loc[idx_,'grid_id']        
                temp_df['grid_id'] = str(new_df.loc[idx_,'grid_id']) + '.' + (temp_df['grid_id'].astype(int)+1).astype(str)
                reassigned_ID.append(temp_df) 
            sub_grids = pd.concat(reassigned_ID)
            sub_grids.reset_index(inplace=True,drop=True)
            sub_grids['upper_id'] = sub_grids['upper_id'].astype(str)
            main_grid_motor_assign['grid_id'] = main_grid_motor_assign['grid_id'].astype(str)
            sub_grids = sub_grids.merge(main_grid_motor_assign[['market_count','motor_count','grid_id']], how='left', left_on='upper_id',right_on='grid_id')
            sub_grids.drop('grid_id_y',axis=1,inplace=True)
            sub_grids.rename(columns={'grid_id_x':'grid_id'},inplace=True)
            
            #Assign markets to grids 
            sub_grid_market = market_df[market_df.grid_id.isin(new_df.grid_id)]
            sub_grid_market = market_to_grid(sub_grid_market,sub_grids)
            
            #Assign orders to grids 
            sub_grid_order = order_df[order_df.grid_id.isin(new_df.grid_id)]
            sub_grid_order = order_to_grid(sub_grid_order, sub_grids) 
            sub_grid_order_ = sub_grid_order.copy()
            
            #Number of orders per subgrid
            sub_grid_order = sub_grid_order.groupby(['grid_id']).agg({'order_ID':'count'}).reset_index()
            sub_grid_order.rename(columns={'order_ID':'order_count'},inplace=True)
            new_sub_grids = sub_grids.merge(sub_grid_order, on='grid_id' , how='left')
            new_sub_grids['order_count'].fillna(0,inplace=True)

            #Number of markets per subgrid
            sub_grid_market_count = sub_grid_market.groupby(['grid_id']).agg({'market_ID':'count'}).reset_index()
            sub_grid_market_count.rename(columns={'market_ID':'market_count'},inplace=True)
            new_sub_grids.drop('market_count',axis=1,inplace=True)
            new_sub_grids = new_sub_grids.merge(sub_grid_market_count, on='grid_id', how='left')
            new_sub_grids['market_count'].fillna(0,inplace=True)
            new_sub_grids = new_sub_grids[new_sub_grids['order_count']!=0]
            new_sub_grids.reset_index(inplace=True,drop=True)      
            
            #Calculate the number of motors to be assigned to the subgrids 
            temp_assigned_motor=[]
            new_sub_grids.upper_id = new_sub_grids.upper_id.astype(str)
            for upperid in new_sub_grids.upper_id.unique():
                temp_df2 = new_sub_grids[new_sub_grids.upper_id==upperid]
                temp_df2 = assign_motor(temp_df2, new_sub_grids[new_sub_grids.upper_id==upperid].motor_count.unique()[0])
                temp_assigned_motor.append(temp_df2)
            subgrid_motor_assigned = pd.concat(temp_assigned_motor)   
            subgrid_motor_assigned.reset_index(inplace=True,drop=True)    
            subsub_grid, idxs_to_drop = grid_division(subgrid_motor_assigned,sub_grid_market,sub_grid_order_,idx_to_drop)
            idx_to_drop = idxs_to_drop + idx_to_drop
            grids=pd.concat([subsub_grid,subgrid_motor_assigned],ignore_index=True)
            main_grid_motor_assign=pd.concat([main_grid_motor_assign,subgrid_motor_assigned],ignore_index=True)    
            if len(main_grid_motor_assign)>1:
                main_grid_motor_assign=pd.concat([main_grid_motor_assign,grids],ignore_index=True)
            else:
                main_grid_motor_assign=grids.copy()
        else:
            continue
    
    return main_grid_motor_assign, list(dict.fromkeys(idx_to_drop))
    

In [25]:
main_grid_motor_assign, idx_to_drop = grid_division(main_grid_motor_assign,city_markets,city_orders,[])


In [26]:
main_grid_motor_assign = main_grid_motor_assign[~main_grid_motor_assign.grid_id.isin(idx_to_drop)]
main_grid_motor_assign.reset_index(drop=True,inplace=True)
main_grid_motor_assign.drop(columns=['order_count'],inplace=True)
main_grid_motor_assign = main_grid_motor_assign[main_grid_motor_assign.motor_count!=0] 

### Assign orders and markets to new grids

In [27]:
city_orders = order_to_grid(city_orders,main_grid_motor_assign)

In [28]:
city_markets = market_to_grid(city_markets, main_grid_motor_assign)

### Assign out of scope orders to the nearest grid
#### The centroids of the grids should be calculated and assigned to the nearest

In [29]:
main_grid_motor_assign['centroid_lat'] = (main_grid_motor_assign['lat_start'] + main_grid_motor_assign['lat_end']) / 2
main_grid_motor_assign['centroid_lon'] = (main_grid_motor_assign['lon_start'] + main_grid_motor_assign['lon_end']) / 2

In [30]:
%%time
for idx in city_orders[city_orders.grid_id=='out_of_scope'].index:
    order_grid_id = city_orders.loc[idx, 'grid_id']
    order_x = city_orders.loc[idx, 'customer_coor_x']
    order_y = city_orders.loc[idx, 'customer_coor_y']
    min_dist = 100000000
    new_grid_id = 0
    for grid_id_ in main_grid_motor_assign.index:
        dist_ = distance.geodesic((main_grid_motor_assign.loc[grid_id_,'centroid_lat'],main_grid_motor_assign.loc[grid_id_,'centroid_lon']), (order_x, order_y)).kilometers
        if(dist_ < min_dist):
            min_dist = dist_
            new_grid_id = main_grid_motor_assign.loc[grid_id_,'grid_id']
    city_orders.loc[idx, 'grid_id'] = new_grid_id
    

CPU times: user 16.5 s, sys: 75 ms, total: 16.6 s
Wall time: 16.6 s


<a id="5"> </a>
### Assign motors to the markets within grids


In [31]:
#The motor count in the grids where the markets are located has been added to the city_markets df and those with 0 motor count has been eliminated
city_markets_ = city_markets.merge(main_grid_motor_assign[['grid_id','motor_count']],how='left',on='grid_id')
city_markets_.fillna(0, inplace=True)
city_markets_ = city_markets_[city_markets_.motor_count!=0]

In [32]:
city_grid_market_list = city_markets_.groupby('grid_id').agg({'market_ID':'unique','motor_count':'unique'})
city_grid_market_list.reset_index(inplace=True)

In [33]:
city_grid_market_list['motor_count'] = city_grid_market_list['motor_count'].apply(lambda x: x[0])

In [34]:
def calculate_geodesic_distance(lat1,lon1,lat2,lon2):
    return (distance.geodesic((lat1,lon1), (lat2,lon2)).kilometers)

In [35]:
def calculate_comb(city_grid_market_list,city_orders_market_gridded):
    assign_motors_final = {}
    for idx in city_grid_market_list.index:
        market_grid_id = city_grid_market_list.loc[idx,'grid_id']
        grid_market_list = city_grid_market_list.loc[idx,'market_ID']
        combinations_object = itertools.combinations(grid_market_list,int(city_grid_market_list.loc[idx,'motor_count']))
        combination_lists = list(combinations_object)
        temp_order = city_orders_market_gridded[city_orders_market_gridded.grid_id== market_grid_id]
        assign_motors = {}
        grid_min_dist = 1000000000000000000000000
        for market_comb in tqdm(combination_lists):
            market_comb_dist_sum = 0
            temp_order['vectorial_dist'] = 1000000000000000000000000
            for markets in market_comb:
                market_x = city_markets[city_markets.market_ID == markets].market_coor_x.values[0]
                market_y = city_markets[city_markets.market_ID == markets].market_coor_y.values[0]
                for order_ in temp_order.index:
                    temp_dist = calculate_geodesic_distance(market_x,market_y,temp_order.loc[order_,'customer_coor_x'],temp_order.loc[order_,'customer_coor_y'])
                    if(temp_dist < temp_order.loc[order_,'vectorial_dist']): #is it minimum distance for the order?
                        temp_order.loc[order_,'vectorial_dist'] = temp_dist
            market_comb_dist_sum = np.sum(temp_order['vectorial_dist'])
            if(market_comb_dist_sum<grid_min_dist):
                grid_min_dist = market_comb_dist_sum
                grid_market_comb = market_comb
        print(market_grid_id, grid_market_comb)
        assign_motors[market_grid_id] = grid_market_comb
        assign_motors_final.update(assign_motors)
    return assign_motors_final  

In [36]:
%%time
assign_results = calculate_comb(city_grid_market_list, city_orders)

100%|█████████████████████████████████████████| 210/210 [00:13<00:00, 15.79it/s]


16.0 (1409, 1410, 1382, 1562)


100%|███████████████████████████████████████████| 35/35 [00:03<00:00,  9.56it/s]


17.0 (1302, 1301, 1567, 1002)


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  9.38it/s]


...

77.0.3.1.4 (1073,)


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 101.07it/s]


77.0.4 (1235,)


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 28.93it/s]

78.0 (1232, 1233, 1536, 1234)
CPU times: user 32min 48s, sys: 11.5 s, total: 32min 59s
Wall time: 33min 1s


In [37]:
%%time
market_district_motor = pd.DataFrame(columns = ['grid_id', 'market_ID','district'])
for key in assign_results:
    for market in assign_results[key]:
        for idx in city_markets.index:
            if (city_markets.loc[idx,'market_ID']==market):
                market_district_motor.loc[idx,'grid_id'] = key
                market_district_motor.loc[idx,'market_ID'] = market
                market_district_motor.loc[idx,'district'] = city_markets.loc[idx,'district']
market_district_motor_count = market_district_motor.groupby('district').agg({'market_ID':'count'})
market_district_motor_count.rename(columns={'market_ID':'motor_count'},inplace=True)
market_district_motor_count.reset_index(inplace=True)

CPU times: user 373 ms, sys: 5.67 ms, total: 379 ms
Wall time: 387 ms


In [38]:
market_motor_assigned = market_district_motor.drop(columns= ['grid_id'])


In [39]:
market_district_motor_count.head()

,district,motor_count
0,10000,3
1,10001,1
2,10002,1
3,10003,1
4,10004,4


In [40]:
market_motor_assigned.head()

,market_ID,district
549,1409,10018
551,1410,10018
553,1382,10018
556,1562,10021
542,1302,10022


In [41]:
market_motor_assigned.shape

(106, 2)

In [42]:
market_district_motor_count.motor_count.sum()

106